In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv('insurance.csv', sep=',')

In [30]:
data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [37]:
col_trans = ColumnTransformer([
    ('hot_encode_sex', OneHotEncoder(handle_unknown='ignore', drop='first'), ['sex', 'smoker', 'region'])
])
pipe = Pipeline([('col_trans', col_trans)])
pipe.fit_transform(data).shape

(1338, 5)

**ADD COLLUMN TRANSFORMER TO SCALE AND TRANSFORM DESCRETE COLUMNS; ADD PIPELINE**

In [13]:
col_trans = ColumnTransformer([
    ('st_scller', StandardScaler(), ('age', 'bmi', 'children')), 
    ('hot_encode', OneHotEncoder(handle_unknown='ignore'), ('sex', 'smoker', 'region'))
])
pipe = Pipeline([('col_trans', col_trans), ('poly_trans', PolynomialFeatures()), ('lasso', Lasso())])

In [14]:
col_trans.fit_transform(data)

array([[1., 0., 0., ..., 0., 0., 1.],
       [0., 1., 1., ..., 0., 1., 0.],
       [0., 1., 1., ..., 0., 1., 0.],
       ...,
       [1., 0., 1., ..., 0., 1., 0.],
       [1., 0., 1., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.]])

In [6]:
pipe.fit(data.loc[:, ~data.columns.isin(['charges'])], data['charges'])

D:\Programming\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.240e+08, tolerance: 1.961e+07
  model = cd_fast.enet_coordinate_descent(


Pipeline(steps=[('col_trans',
                 ColumnTransformer(transformers=[('st_scller', StandardScaler(),
                                                  ('age', 'bmi', 'children')),
                                                 ('hot_encode',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ('sex', 'smoker',
                                                   'region'))])),
                ('poly_trans', PolynomialFeatures()), ('lasso', Lasso())])

In [7]:
pipe.predict(data.loc[:, ~data.columns.isin(['charges'])])

array([24261.94120101,  3239.15258969,  5996.9567249 , ...,
        2247.38902164,  3000.7083045 , 35554.60153108])

**APPLY GRID SEARCH TO SELECT HYPERPARAMETERS**

In [8]:
%%time
params = dict(
    lasso__alpha= np.linspace(0.001, 10, 100), 
    poly_trans__degree = [2, 3, ], 
    poly_trans__include_bias = [False, ], 
)
alfa_search = GridSearchCV(pipe, params, scoring='neg_mean_absolute_error', verbose=1, n_jobs=8)
alfa_search.fit(data.loc[:, ~data.columns.isin(['charges'])], data['charges'])

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Wall time: 12.4 s


GridSearchCV(estimator=Pipeline(steps=[('col_trans',
                                        ColumnTransformer(transformers=[('st_scller',
                                                                         StandardScaler(),
                                                                         ('age',
                                                                          'bmi',
                                                                          'children')),
                                                                        ('hot_encode',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ('sex',
                                                                          'smoker',
                                                                          'region'))])),
                                       ('poly_trans', PolynomialFeatures()),
                                       ('lasso', Lasso())]),
             n_jobs=8,
             param_grid={'lasso__alpha': array([1.000e-03, 1.020e-01, 2.030e-0...
       7.273e+00, 7.374e+00, 7.475e+00, 7.576e+00, 7.677e+00, 7.778e+00,
       7.879e+00, 7.980e+00, 8.081e+00, 8.182e+00, 8.283e+00, 8.384e+00,
       8.485e+00, 8.586e+00, 8.687e+00, 8.788e+00, 8.889e+00, 8.990e+00,
       9.091e+00, 9.192e+00, 9.293e+00, 9.394e+00, 9.495e+00, 9.596e+00,
       9.697e+00, 9.798e+00, 9.899e+00, 1.000e+01]),
                         'poly_trans__degree': [2, 3],
                         'poly_trans__include_bias': [False]},
             scoring='neg_mean_absolute_error', verbose=1)

In [9]:
alfa_search.best_score_

-2947.548490762149

In [10]:
alfa_search.best_score_

-2947.548490762149

# **ДЗ**

**1. Сделать это только предварительно выбрать валидационный базис**
**2. Сделать то же самое для дерева (пайп и grid search)**
**3. Из матрицы результатов, получаемых при разных гиперпараметрах, посмотреть какие наиболее значимые**
**4. Сделать то же для случайного леса (пайп и grid search)**